******************************************************************************************************************************* 
NOTE SUR LE PROJET: Michaël, votre manager, vous incite à sélectionner un ou des kernels Kaggle pour vous faciliter l’analyse exploratoire, la préparation des données et le feature engineering nécessaires à l’élaboration du modèle de scoring. Si vous le faites, vous devez analyser ce ou ces kernels et le ou les adapter pour vous assurer qu’il(s) répond(ent) aux besoins de votre mission. C’est **optionnel**, mais nous vous encourageons à le faire afin de vous permettre de vous focaliser sur l’élaboration du modèle, son optimisation et sa compréhension.                            

Comme ce n'était pas obligatoire, je ne suis pas partie du kernel de Kaggle dans un souhait d'apprentissage. En effet, étant intéressée par ce domaine et mon mentor étant expert dans ce domaine, j'ai eu l'opportunité de réaliser ce projet en immersion dans une agence bancaire.

Même si cela n'était pas demandé, j'ai réalisé une EDA ce qui est de mon point de vue indispensable pour la compréhension des données en notre possession.

Le processing et le feature engineering de la table 'previous_application' seront réalisés dans ce notebook. Certaines lignes de code prenant plus de 4 heures d'exécution, j'ai été dans l'obligation de scinder mes notebooks. Le prochain notebook sera dédié aux tables 'credit_card_balance', 'installments_payments' et 'POS_CASH_balance'.
*******************************************************************************************************************************

# IMPLEMENTEZ UN MODELE DE SCORING

# Création du modèle de scoring

### Contexte

Vous êtes Data Scientist au sein d'une société financière, nommée "Prêt à dépenser", qui propose des crédits à la consommation pour des personnes ayant peu ou pas du tout d'historique de prêt.

L’entreprise souhaite mettre en œuvre un outil de “scoring crédit” pour calculer la probabilité qu’un client rembourse son crédit, puis classifier la demande en crédit accordé ou refusé. Elle souhaite donc développer un algorithme de classification en s’appuyant sur des sources de données variées (données comportementales, données provenant d'autres institutions financières, etc.).

De plus, les chargés de relation client ont fait remonter le fait que les clients sont de plus en plus demandeurs de transparence vis-à-vis des décisions d’octroi de crédit. Cette demande de transparence des clients va tout à fait dans le sens des valeurs que l’entreprise veut incarner.

Prêt à dépenser décide donc de développer un dashboard interactif pour que les chargés de relation client puissent à la fois expliquer de façon la plus transparente possible les décisions d’octroi de crédit, mais également permettre à leurs clients de disposer de leurs informations personnelles et de les explorer facilement. 

### Missions
- **Mission 1: Construction d'un modèle de scoring donnant une prédiction sur la probabilité de faillite d'un client de façon automatique.**
- **Mission 2: Construction d'un dashboard interactif à destination des gestionnaires de la relation client permettant d'interpréter les prédictions faites par le modèle, et d’améliorer la connaissance client des chargés de relation client.**
- **Mission 3: Mise en production du modèle de scoring de prédiction à l’aide d’une API, ainsi que du dashboard interactif appelant l’API pour les prédictions.**

In [1]:
# Import des librairies classiques pour l'EDA

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mode

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# 1. Le jeu de données previous_application

## 1.1. Ouverture du fichier et informations principales

In [2]:
# Ouverture du fichier
file_1 = pd.read_csv("previous_application.csv", sep=",")
pd.set_option("Display.max_rows", None)
pd.set_option("Display.max_columns", None)
file_1.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.0,0.182832,0.867336,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,Y,1,NaN,NaN,NaN,XNA,Approved,-512,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,Y,1,NaN,NaN,NaN,Repairs,Refused,-781,Cash through the bank,HC,NaN,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Copie du fichier
previous_application = file_1.copy()

In [4]:
# Informations sur le jeu de données
previous_application.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1670214 entries, 0 to 1670213
Data columns (total 37 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   SK_ID_PREV                   1670214 non-null  int64  
 1   SK_ID_CURR                   1670214 non-null  int64  
 2   NAME_CONTRACT_TYPE           1670214 non-null  object 
 3   AMT_ANNUITY                  1297979 non-null  float64
 4   AMT_APPLICATION              1670214 non-null  float64
 5   AMT_CREDIT                   1670213 non-null  float64
 6   AMT_DOWN_PAYMENT             774370 non-null   float64
 7   AMT_GOODS_PRICE              1284699 non-null  float64
 8   WEEKDAY_APPR_PROCESS_START   1670214 non-null  object 
 9   HOUR_APPR_PROCESS_START      1670214 non-null  int64  
 10  FLAG_LAST_APPL_PER_CONTRACT  1670214 non-null  object 
 11  NFLAG_LAST_APPL_IN_DAY       1670214 non-null  int64  
 12  RATE_DOWN_PAYMENT            774370 non-nu

**A NOTER:** Concernant ce jeu de données, toutes les variables ne seront pas conservées. En fonction du nombre de données présentes et de la signification de la variable, seules les variables PERTINENTES et UTILISABLES seront conservées.

## 1.2. Analyse des différentes variables catégorielles d'intérêt

### 1. La variable SK_ID_CURR

In [5]:
# Nombre de clients présents dans la base de données
len(previous_application["SK_ID_CURR"].unique().tolist())

338857

*Cette base de données recense 338857 clients, alors que le Bureau de crédits en référence 305811. Nous avons donc de nouveaux clients.*

### 2. La variable 'NAME_CONTRACT_STATUS'

**1. ANALYSE DE LA VARIABLE**

In [6]:
# Pourcentage de refus et d'acception sans tenir compte du client
previous_application["NAME_CONTRACT_STATUS"].value_counts(normalize=True)

NAME_CONTRACT_STATUS
Approved        0.620747
Canceled        0.189388
Refused         0.174036
Unused offer    0.015828
Name: proportion, dtype: float64

*Certains clients ont annulé ou n'ont pas finalisé leur demande de prêt ce qui en soit signifie que le prêt est inexistant. Ces prêts seront néanmoins conservés afin d'obtenir un meilleur profil du client.*

**2. ENCODAGE MANUEL DE LA VARIABLE**

In [7]:
# Comptage de chaque statut de crédits par client dans une catégorie
credit_demande = previous_application.groupby(["SK_ID_CURR", "NAME_CONTRACT_STATUS"]).agg({"NAME_CONTRACT_STATUS": "count"}).unstack() # unstack = pivot: clients en lignes et catégorie en colonnes
credit_demande.columns = credit_demande.columns.droplevel(0) 
credit_demande.fillna(0, inplace=True),
credit_demande["NAME_CONTRACT_STATUS"] = credit_demande.sum(axis=1)
        
# Elimination de la variable 'NAME_CONTRACT_STATUS' devenue inutile
credit_demande = credit_demande.drop("NAME_CONTRACT_STATUS", axis=1)

credit_demande.reset_index(inplace=True) 
credit_demande.head()

NAME_CONTRACT_STATUS,SK_ID_CURR,Approved,Canceled,Refused,Unused offer
0,100001,1.0,0.0,0.0,0.0
1,100002,1.0,0.0,0.0,0.0
2,100003,3.0,0.0,0.0,0.0
3,100004,1.0,0.0,0.0,0.0
4,100005,1.0,1.0,0.0,0.0


In [8]:
# Description de la variable
credit_demande.describe()

NAME_CONTRACT_STATUS,SK_ID_CURR,Approved,Canceled,Refused,Unused offer
count,338857.000000,338857.000000,338857.000000,338857.000000,338857.000000
mean,278149.909581,3.059642,0.933488,0.857819,0.078015
std,102879.193103,2.135404,1.723006,1.830574,0.327991
min,100001.000000,0.000000,0.000000,0.000000,0.000000
25%,189061.000000,1.000000,0.000000,0.000000,0.000000
50%,278221.000000,3.000000,0.000000,0.000000,0.000000
75%,367302.000000,4.000000,1.000000,1.000000,0.000000
max,456255.000000,27.000000,58.000000,68.000000,21.000000


### 3. La variable 'NAME_CONTRACT_TYPE'

**1. ANALYSE DE LA VARIABLE**

In [9]:
# Contenu de la variable
print("Contenu de la variable 'NAME_CONTRACT_TYPE' dans le jeu de données previous_application")
print(previous_application["NAME_CONTRACT_TYPE"].unique().tolist())

Contenu de la variable 'NAME_CONTRACT_TYPE' dans le jeu de données previous_application
['Consumer loans', 'Cash loans', 'Revolving loans', 'XNA']


In [10]:
# Comptage des valeurs
previous_application["NAME_CONTRACT_TYPE"].value_counts(normalize= True)

NAME_CONTRACT_TYPE
Cash loans         0.447579
Consumer loans     0.436561
Revolving loans    0.115652
XNA                0.000207
Name: proportion, dtype: float64

**A NOTER**: Nous sommes en présence de très peu de valeurs manquantes. Ces dernières seront 'injectées' dans 'Cash loans' car dans les jeux d'entraînement et de test, le type de contrat est soit Cash loans (quasi 95%) soit Revolving loans (quasi 5%), avec les Cash Loans étant davantage à risque (informations connues car travail en parallèle...).

**2. REGROUPEMENT DES TYPES DE CONTRATS PAR CLIENT**

In [11]:
# Regroupement des 'XNA' et 'Cash Loans'
xna = previous_application[(previous_application["NAME_CONTRACT_TYPE"] == "XNA") | (previous_application["NAME_CONTRACT_TYPE"] == "Cash loans")]

for idx in xna.index:
    previous_application.loc[idx, "NAME_CONTRACT_TYPE"] = "Cash loans"

In [12]:
# Regroupement des types de contrats par client
type_contrat_client = previous_application.groupby("SK_ID_CURR")["NAME_CONTRACT_TYPE"].value_counts().unstack().fillna(0)
type_contrat_client.head()

NAME_CONTRACT_TYPE,Cash loans,Consumer loans,Revolving loans
SK_ID_CURR,,,
100001,0.0,1.0,0.0
100002,0.0,1.0,0.0
100003,1.0,2.0,0.0
100004,0.0,1.0,0.0
100005,1.0,1.0,0.0


In [13]:
# Description du jeu de données
type_contrat_client.describe()

NAME_CONTRACT_TYPE,Cash loans,Consumer loans,Revolving loans
count,338857.000000,338857.000000,338857.000000
mean,2.207123,2.151796,0.570046
std,3.298588,1.820743,1.024132
min,0.000000,0.000000,0.000000
25%,0.000000,1.000000,0.000000
50%,1.000000,2.000000,0.000000
75%,3.000000,3.000000,1.000000
max,66.000000,48.000000,31.000000


**3. ENCODAGE MANUEL DE LA VARIABLE**

In [14]:
# Comptage de chaque type de crédits par client dans une catégorie
loan_type = previous_application.groupby(["SK_ID_CURR", "NAME_CONTRACT_TYPE"]).agg({"NAME_CONTRACT_TYPE": "count"}).unstack() # unstack = pivot: clients en lignes et catégorie en colonnes
loan_type.columns = loan_type.columns.droplevel(0) 
loan_type.fillna(0, inplace=True),
loan_type["NAME_CONTRACT_TYPE"] = loan_type.sum(axis=1)
        
# Elimination de la variable 'NAME_CONTRACT_TYPE' devenue inutile
loan_type = loan_type.drop("NAME_CONTRACT_TYPE", axis=1)

loan_type.reset_index(inplace=True) 
loan_type.head()

NAME_CONTRACT_TYPE,SK_ID_CURR,Cash loans,Consumer loans,Revolving loans
0,100001,0.0,1.0,0.0
1,100002,0.0,1.0,0.0
2,100003,1.0,2.0,0.0
3,100004,0.0,1.0,0.0
4,100005,1.0,1.0,0.0


### 4. Jointure des dataframes credit_demande et loan_type

**Cette jointure sera réalisée via la variable 'SK_ID_CURR'.**

In [15]:
# Création du dataframe
merge_1 = pd.merge(credit_demande, loan_type, on="SK_ID_CURR", how="left")
merge_1.head()

,SK_ID_CURR,Approved,Canceled,Refused,Unused offer,Cash loans,Consumer loans,Revolving loans
0,100001,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,100002,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,100003,3.0,0.0,0.0,0.0,1.0,2.0,0.0
3,100004,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,100005,1.0,1.0,0.0,0.0,1.0,1.0,0.0


### 5. La variable 'FLAG_LAST_APPL_PER_CONTRACT'

*Parfois, il peut y avoir plusieurs demandes pour un seul et même contrat, que ce soit en raison d'une erreur du client ou d'une erreur administrative (erreur du commis). Par conséquent, pour éviter la confusion ou pour des raisons d'analyse, il serait utile d'avoir un indicateur qui montre clairement quelle demande était la "dernière" pour un contrat donné.*

In [16]:
previous_application["FLAG_LAST_APPL_PER_CONTRACT"].value_counts()

FLAG_LAST_APPL_PER_CONTRACT
Y    1661739
N       8475
Name: count, dtype: int64

Il n'y a que 8475 demandes qui ne sont pas mises à jour... Cette variable ne sera donc pas utilisée dans le projet car en plus d'être quasiment à jour, elle ne constitue en rien le fait que le client soit à risque ou non.

### 6. La variable NAME_CASH_LOAN_PURPOSE

In [17]:
# Contenu de la variable
print(previous_application["NAME_CASH_LOAN_PURPOSE"].unique().tolist())

['XAP', 'XNA', 'Repairs', 'Everyday expenses', 'Car repairs', 'Building a house or an annex', 'Other', 'Journey', 'Purchase of electronic equipment', 'Medicine', 'Payments on other loans', 'Urgent needs', 'Buying a used car', 'Buying a new car', 'Buying a holiday home / land', 'Education', 'Buying a home', 'Furniture', 'Buying a garage', 'Business development', 'Wedding / gift / holiday', 'Hobby', 'Gasification / water supply', 'Refusal to name the goal', 'Money for a third person']


In [18]:
# Pourcentage de données pour chaque type de demandes
previous_application["NAME_CASH_LOAN_PURPOSE"].value_counts(normalize=True)

NAME_CASH_LOAN_PURPOSE
XAP                                 0.552421
XNA                                 0.405887
Repairs                             0.014229
Other                               0.009345
Urgent needs                        0.005036
Buying a used car                   0.001729
Building a house or an annex        0.001612
Everyday expenses                   0.001447
Medicine                            0.001302
Payments on other loans             0.001156
Education                           0.000942
Journey                             0.000742
Purchase of electronic equipment    0.000635
Buying a new car                    0.000606
Wedding / gift / holiday            0.000576
Buying a home                       0.000518
Car repairs                         0.000477
Furniture                           0.000448
Buying a holiday home / land        0.000319
Business development                0.000255
Gasification / water supply         0.000180
Buying a garage                 

**Comme nous pouvons le remarquer, cette variable sera INUTILISABLE car elle totalise quasi 95% de valeurs manquantes.**

### 7. La variable 'NAME_PAYMENT_TYPE'

In [19]:
# Contenu de la variable
print(previous_application["NAME_PAYMENT_TYPE"].unique().tolist())

['Cash through the bank', 'XNA', 'Non-cash from your account', 'Cashless from the account of the employer']


In [20]:
# Pourcentage de chaque donnée
previous_application["NAME_PAYMENT_TYPE"].value_counts(normalize=True)

NAME_PAYMENT_TYPE
Cash through the bank                        0.618814
XNA                                          0.375631
Non-cash from your account                   0.004905
Cashless from the account of the employer    0.000650
Name: proportion, dtype: float64

**Plus de 98% des données sont regroupées dans 2 catégories, 'Cash through the bank' (plus de 75% des données) et  valeurs manquantes (24%), rendant donc cette variable INUTILISABLE.**

### 8. La variable 'CODE_REJECT_REASON'

**Cette variable pourrait être intéressante si elle présente suffisamment de données.**

In [21]:
# Contenu de la variable
print(previous_application["CODE_REJECT_REASON"].unique().tolist())

['XAP', 'HC', 'LIMIT', 'CLIENT', 'SCOFR', 'SCO', 'XNA', 'VERIF', 'SYSTEM']


In [22]:
# Pourcentage de chaque donnée
previous_application["CODE_REJECT_REASON"].value_counts(normalize=True)

CODE_REJECT_REASON
XAP       0.810132
HC        0.104915
LIMIT     0.033337
SCO       0.022432
CLIENT    0.015828
SCOFR     0.007670
XNA       0.003140
VERIF     0.002116
SYSTEM    0.000429
Name: proportion, dtype: float64

**Même si les 'XAP' correspondent aux demandes acceptées et qu'il y a de ce fait très peu de données manquantes pour les crédits refusés, vu le contenu et le nombre très important de variables dont nous sommes déjà en possession, cette dernière ne sera pas incluse dans le projet. En effet, la principale information est de savoir si le prêt a été accepté ou refusé...**

### 9. La variable 'NAME_TYPE_SUITE'

In [23]:
# Contenu de la variable
print(previous_application["NAME_TYPE_SUITE"].unique().tolist())

[nan, 'Unaccompanied', 'Spouse, partner', 'Family', 'Children', 'Other_B', 'Other_A', 'Group of people']


In [24]:
# Nombre de valeurs manquantes
missing_accompagnement = previous_application[previous_application["NAME_TYPE_SUITE"].isna()]
print(f"Nous n\'avons pas de données pour {missing_accompagnement.shape[0]} lignes.")
print("-------------------------------------------------------------------------------------")
# Nombre de valeurs présentes
data_accompagnement = previous_application[previous_application["NAME_TYPE_SUITE"].notna()]
print(f"Nous avons des données pour {data_accompagnement.shape[0]} lignes.")       

Nous n'avons pas de données pour 820405 lignes.
-------------------------------------------------------------------------------------
Nous avons des données pour 849809 lignes.


**Dans le contexte d'octroi de crédit, cette donnée peut être utile. En effet, l'accompagnement d'une personne pourrait être une source de 'garantie'. Nous sommes en présence d'environ 50 % de données et afin d'éviter de perdre de l'information, nous pouvons essayer de répartir les données en 3 types de catégories: 'Unknown', 'Accompanied' et 'Unaccompanied'.**

**Nous allons donc regarder si cela est envisageable en terme de répartition des données.**

In [25]:
# Création de la donnée 'Unknown'
previous_application["NAME_TYPE_SUITE"] = previous_application["NAME_TYPE_SUITE"].fillna(value='Unknown')

# Création de la donnée 'Accompanied'
accompanied= previous_application[(previous_application["NAME_TYPE_SUITE"]== "Spouse, partner") | (previous_application["NAME_TYPE_SUITE"]== "Family") |
                                 (previous_application["NAME_TYPE_SUITE"]== "Children") | (previous_application["NAME_TYPE_SUITE"]== "Other_B") |
                                 (previous_application["NAME_TYPE_SUITE"]== "Other_A") | (previous_application["NAME_TYPE_SUITE"]== "Group of people")]

for idx in accompanied.index:
    previous_application.loc[idx, "NAME_TYPE_SUITE"] = "Accompanied"

In [26]:
# Pourcentage de données par catégorie
previous_application["NAME_TYPE_SUITE"].value_counts(normalize=True)

NAME_TYPE_SUITE
Unknown          0.491198
Unaccompanied    0.304733
Accompanied      0.204069
Name: proportion, dtype: float64

**Etant en présence de 3 catégories équilibrées, un encodage manuel sera réalisé.**

**ENCODAGE MANUEL POUR L'ACCOMPAGNEMENT OU NON DU CLIENT LORS DE LA DEMANDE DE PRET**

In [27]:
# Création d'un dataframe accompagnement_client 
accompagnement_client = previous_application[["SK_ID_CURR", "NAME_TYPE_SUITE"]]


# Encodage "Unknown": -1, "Unaccompanied": 0, "Accompanied": 1
accompagnement_client["NAME_TYPE_SUITE"] = accompagnement_client["NAME_TYPE_SUITE"].replace({"Unknown": -1, "Unaccompanied": 0, 
                                                                                           "Accompanied": 1})

accompagnement_client.head()

C:\Users\colin\AppData\Local\Temp\ipykernel_16592\2920412914.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  accompagnement_client["NAME_TYPE_SUITE"] = accompagnement_client["NAME_TYPE_SUITE"].replace({"Unknown": -1, "Unaccompanied": 0,


,SK_ID_CURR,NAME_TYPE_SUITE
0,271877,-1
1,108129,0
2,122040,1
3,176158,-1
4,202054,-1


**Un client pouvant avoir fait plusieurs demandes, il se peut qu'il soit accompagné ou non lors de ces dernières. De ce fait, l'agrégation sera réalisée sur le 'plus fréquent' via la fonction .mode().**

In [28]:
aggregated_accompagnement_client = accompagnement_client.groupby('SK_ID_CURR')['NAME_TYPE_SUITE'].agg(lambda x: x.mode().iloc[0]).reset_index()
aggregated_accompagnement_client.head()

,SK_ID_CURR,NAME_TYPE_SUITE
0,100001,1
1,100002,-1
2,100003,1
3,100004,0
4,100005,-1


### 10. Jointure des dataframes merge_1 et aggregated_accompagnement_client

**La jointure sera réalisée via la variable 'SK_ID_CURR'.**

In [29]:
merge_2 = pd.merge(merge_1, aggregated_accompagnement_client, on='SK_ID_CURR', how= 'left')
merge_2.head()

,SK_ID_CURR,Approved,Canceled,Refused,Unused offer,Cash loans,Consumer loans,Revolving loans,NAME_TYPE_SUITE
0,100001,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1
1,100002,1.0,0.0,0.0,0.0,0.0,1.0,0.0,-1
2,100003,3.0,0.0,0.0,0.0,1.0,2.0,0.0,1
3,100004,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0
4,100005,1.0,1.0,0.0,0.0,1.0,1.0,0.0,-1


### 11. La variable 'NAME_CLIENT_TYPE'

In [30]:
# Contenu de la variable
print(previous_application["NAME_CLIENT_TYPE"].unique().tolist())

['Repeater', 'New', 'Refreshed', 'XNA']


In [31]:
# Nombre de valeurs manquantes
missing_type_client = previous_application[previous_application["NAME_CLIENT_TYPE"] == "XNA"]
print(f"Nous n\'avons pas de données pour {missing_type_client.shape[0]} lignes.")

Nous n'avons pas de données pour 1941 lignes.


*Même si cette donnée peut être intéressante, il s'agit d'une variable catégorielle ce qui amplifierait davantage notre nombre de variables, avec en plus des données inconnues. De mon point de vue, pour l'octroi d'un crédit, cette donnée n'est pas indispensable. Par conséquent, cette variable ne sera pas prise en compte.*

### 12. La variable 'NAME_GOODS_CATEGORY'

In [32]:
# Contenu de la variable
print(previous_application["NAME_GOODS_CATEGORY"].unique().tolist())

['Mobile', 'XNA', 'Consumer Electronics', 'Construction Materials', 'Auto Accessories', 'Photo / Cinema Equipment', 'Computers', 'Audio/Video', 'Medicine', 'Clothing and Accessories', 'Furniture', 'Sport and Leisure', 'Homewares', 'Gardening', 'Jewelry', 'Vehicles', 'Education', 'Medical Supplies', 'Other', 'Direct Sales', 'Office Appliances', 'Fitness', 'Tourism', 'Insurance', 'Additional Service', 'Weapon', 'Animals', 'House Construction']


In [33]:
# Nombre de valeurs manquantes et présentes
missing_goods = previous_application[previous_application["NAME_GOODS_CATEGORY"] == "XNA"]
print(f"Nous n\'avons pas de données pour {missing_goods.shape[0]} lignes.")
print("-----------------------------------------------------------------------------------")
precised_goods = previous_application[previous_application["NAME_GOODS_CATEGORY"] != "XNA"]
print(f"Nous avons des données pour {precised_goods.shape[0]} lignes.")

Nous n'avons pas de données pour 950809 lignes.
-----------------------------------------------------------------------------------
Nous avons des données pour 719405 lignes.


*Ayant 57% de données et beaucoup de catégories dont une 'other', nous pourrions envisager de réaliser des regroupements afin de réduire le nombre de variables. Toutefois, dans ce genre de variables, il y a souvent des erreurs réalisées, amoindrissant donc sa fiabilité. De plus, je ne pense pas que cette dernière soit un critère décisif dans la classification du client à risque ou non.*

**Par conséquent, cette variable ne sera pas prise en compte.**

### 13. La variable 'NFLAG_INSURED_ON_APPROVAL'

*Cette variable renseigne sur le fait qu'un client a demandé une assurance ou non pour le crédit précédent. Cela peut souligner qu'il se considère comme ayant un certain risque de ne pas être en mesure de rembourser le prêt.*

In [34]:
# Contenu de la variable
print(previous_application["NFLAG_INSURED_ON_APPROVAL"].unique().tolist())

[0.0, 1.0, nan]


In [35]:
# Nombre de valeurs manquantes et présentes
missing_insured = previous_application[previous_application["NFLAG_INSURED_ON_APPROVAL"].isna()]
print(f"Nous n\'avons pas de données pour {missing_insured.shape[0]} lignes.")
print("-----------------------------------------------------------------------------------")
precised_insured = previous_application[previous_application["NFLAG_INSURED_ON_APPROVAL"].notna()]
print(f"Nous avons des données pour {precised_insured.shape[0]} lignes.")

Nous n'avons pas de données pour 673065 lignes.
-----------------------------------------------------------------------------------
Nous avons des données pour 997149 lignes.


**Même si nous sommes en présence de 40% de données manquantes, cette variable sera prise en compte. Les valeurs manquantes seront remplacées par -1 et l'agrégation sera réalisée sur la valeur la plus fréquente.**

In [36]:
# Remplacement des nan par -1
previous_application["NFLAG_INSURED_ON_APPROVAL"] = previous_application["NFLAG_INSURED_ON_APPROVAL"].fillna(value=-1)

In [37]:
aggregated_assurance_client = previous_application.groupby('SK_ID_CURR')['NFLAG_INSURED_ON_APPROVAL'].agg(lambda x: x.mode().iloc[0]).reset_index()
aggregated_assurance_client.head()

,SK_ID_CURR,NFLAG_INSURED_ON_APPROVAL
0,100001,0.0
1,100002,0.0
2,100003,1.0
3,100004,0.0
4,100005,-1.0


### 14. Jointure des dataframes merge_2 et aggregated_assurance_client

In [38]:
# La jointure en elle-même
aggregated_previous_categorical = pd.merge(merge_2, aggregated_assurance_client, on='SK_ID_CURR', how='left')
aggregated_previous_categorical.head()

,SK_ID_CURR,Approved,Canceled,Refused,Unused offer,Cash loans,Consumer loans,Revolving loans,NAME_TYPE_SUITE,NFLAG_INSURED_ON_APPROVAL
0,100001,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0.0
1,100002,1.0,0.0,0.0,0.0,0.0,1.0,0.0,-1,0.0
2,100003,3.0,0.0,0.0,0.0,1.0,2.0,0.0,1,1.0
3,100004,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0.0
4,100005,1.0,1.0,0.0,0.0,1.0,1.0,0.0,-1,-1.0


In [39]:
# Vérification du nombre de clients (attendu 338857)
print(f'Notre jeu de données comprend bien {aggregated_previous_categorical.shape[0]} clients.')

Notre jeu de données comprend bien 338857 clients.


In [40]:
# Renommage des colonnes pour plus de clarté
aggregated_previous_categorical= aggregated_previous_categorical.rename(columns={"Approved":"prev_approved", "Canceled": "prev_canceled",
                                                                                "Refused":"prev_refused", "Unused offer":"prev_unused_offer",
                                                                                "Cash loans":"prev_cash_loans", "Consumer loans":"prev_consumer_loans",
                                                                                "Revolving loans":"prev_revolving_loans", "NAME_TYPE_SUITE":"prev_NAME_TYPE_SUITE",
                                                                                "NFLAG_INSURED_ON_APPROVAL":"prev_NFLAG_INSURED_ON_APPROVAL"})
aggregated_previous_categorical.head()

,SK_ID_CURR,prev_approved,prev_canceled,prev_refused,prev_unused_offer,prev_cash_loans,prev_consumer_loans,prev_revolving_loans,prev_NAME_TYPE_SUITE,prev_NFLAG_INSURED_ON_APPROVAL
0,100001,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0.0
1,100002,1.0,0.0,0.0,0.0,0.0,1.0,0.0,-1,0.0
2,100003,3.0,0.0,0.0,0.0,1.0,2.0,0.0,1,1.0
3,100004,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0.0
4,100005,1.0,1.0,0.0,0.0,1.0,1.0,0.0,-1,-1.0


## 1.3. Analyse des différentes variables numériques d'intérêt

**A NOTER: Très peu de variables numériques seront retenues soit par manque d'utilité dans notre projet, soit par la présence de trop de données manquantes.**
- **Variables non pertinentes**:NFLAG_LAST_APPL_IN_DAY, NFLAG_MICRO_CASH, 
- **Variables avec peu de données**: RATE_INTEREST_PRIMARY, RATE_INTEREST_PRIVILEGED

**TRES IMPORTANT: Concernant les variables numériques, il sera indispensable d'être vigilant que sur le fait ces dernières doivent être impérativement de zéro lorsque le crédit est 'refused', 'canceled' or 'unused_offer'. Si tel n'est pas le cas, elle seront modifiées afin de ne pas introduire un gros biais dans les données.**

### 1. Mise à zéro des variables 'AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT' et 'AMT_GOODS_PRICE' pour les crédits refusés, annulés et non signés

**En effet, les crédits étant refusés, afin d'éviter de fausser les données lors de l'agrégation, ces variables seront mises à 0...**

In [41]:
condition = ((aggregated_previous_categorical["prev_refused"] != 0 ) | (aggregated_previous_categorical["prev_canceled"] !=0) |
             (aggregated_previous_categorical["prev_unused_offer"] != 0)) & (aggregated_previous_categorical["prev_approved"]==0)

refused_loan = aggregated_previous_categorical[condition]
refused_loan.head()

,SK_ID_CURR,prev_approved,prev_canceled,prev_refused,prev_unused_offer,prev_cash_loans,prev_consumer_loans,prev_revolving_loans,prev_NAME_TYPE_SUITE,prev_NFLAG_INSURED_ON_APPROVAL
113,100120,0.0,3.0,0.0,0.0,3.0,0.0,0.0,-1,-1.0
475,100505,0.0,0.0,1.0,0.0,1.0,0.0,0.0,-1,-1.0
582,100614,0.0,1.0,0.0,1.0,0.0,1.0,1.0,-1,-1.0
991,101040,0.0,3.0,3.0,0.0,6.0,0.0,0.0,-1,-1.0
1234,101291,0.0,0.0,3.0,0.0,0.0,3.0,0.0,0,-1.0


In [42]:
print(f"{refused_loan.shape[0]} clients n\'ont pas de prêt accordé dans ce jeu de données.")

1159 clients n'ont pas de prêt accordé dans ce jeu de données.


In [43]:
refused_loan = previous_application[previous_application["NAME_CONTRACT_STATUS"] != "Approved"]
refused_loan.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,Y,1,NaN,NaN,NaN,Repairs,Refused,-781,Cash through the bank,HC,Unknown,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,-1.0
6,2315218,175704,Cash loans,NaN,0.0,0.0,NaN,NaN,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Canceled,-14,XNA,XAP,Unknown,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,-1.0
7,1656711,296299,Cash loans,NaN,0.0,0.0,NaN,NaN,MONDAY,7,Y,1,NaN,NaN,NaN,XNA,Canceled,-21,XNA,XAP,Unknown,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,-1.0
8,2367563,342292,Cash loans,NaN,0.0,0.0,NaN,NaN,MONDAY,15,Y,1,NaN,NaN,NaN,XNA,Canceled,-386,XNA,XAP,Unknown,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,-1.0
9,2579447,334349,Cash loans,NaN,0.0,0.0,NaN,NaN,SATURDAY,15,Y,1,NaN,NaN,NaN,XNA,Canceled,-57,XNA,XAP,Unknown,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,-1.0


In [44]:
# Remplacement par 0 dans le jeu de données

refused_loan = refused_loan.copy()

for idx in refused_loan.index:
    previous_application.loc[idx, "AMT_ANNUITY"] = 0
    previous_application.loc[idx, "AMT_APPLICATION"] = 0
    previous_application.loc[idx, "AMT_CREDIT"] = 0
    previous_application.loc[idx, "AMT_DOWN_PAYMENT"] = 0
    previous_application.loc[idx, "AMT_GOODS_PRICE"] = 0

In [45]:
# Vérification de la bonne imputation sur les 10ères lignes
previous_application.head(10)

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.0,0.182832,0.867336,XAP,Approved,-73,Cash through the bank,XAP,Unknown,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-301,Cash through the bank,XAP,Accompanied,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,Y,1,NaN,NaN,NaN,XNA,Approved,-512,Cash through the bank,XAP,Unknown,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,0.000,0.0,0.0,0.0,0.0,THURSDAY,9,Y,1,NaN,NaN,NaN,Repairs,Refused,-781,Cash through the bank,HC,Unknown,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,-1.0
5,1383531,199383,Cash loans,23703.930,315000.0,340573.5,NaN,315000.0,SATURDAY,8,Y,1,NaN,NaN,NaN,Everyday expenses,Approved,-684,Cash through the bank,XAP,Accompanied,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,18.0,low_normal,Cash X-Sell: low,365243.0,-654.0,-144.0,-144.0,-137.0,1.0
6,2315218,175704,Cash loans,0.000,0.0,0.0,0.0,0.0,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Canceled,-14,XNA,XAP,Unknown,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,-1.0
7,1656711,296299,Cash loans,0.000,0.0,0.0,0.0,0.0,MONDAY,7,Y,1,NaN,NaN,NaN,XNA,Canceled,-21,XNA,XAP,Unknown,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,-1.0
8,2367563,342292,Cash loans,0.000,0.0,0.0,0.0,0.0,MONDAY,15,Y,1,NaN,NaN,NaN,XNA,Canceled,-386,XNA,XAP,Unknown,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,-1.0
9,2579447,334349,Cash loans,0.000,0.0,0.0,0.0,0.0,SATURDAY,15,Y,1,NaN,NaN,NaN,XNA,Canceled,-57,XNA,XAP,Unknown,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,-1.0


### 2. La variable AMT_ANNUITY

**A NOTER**: Présence de valeurs manquantes. Comme pour le jeu de données du Bureau de crédits, une imputation par la valeur fictive de -2 sera réalisée.

In [46]:
# Description de la variable
previous_application["AMT_ANNUITY"].describe()

count    1.670206e+06
mean     9.137592e+03
std      1.301031e+04
min      0.000000e+00
25%      0.000000e+00
50%      5.126040e+03
75%      1.271326e+04
max      3.938687e+05
Name: AMT_ANNUITY, dtype: float64

In [47]:
# Imputation des valeurs manquantes par la valeur fictive de -2
previous_application["AMT_ANNUITY"] = previous_application["AMT_ANNUITY"].fillna(value=-2)

*Comme pour le jeu de données du Bureau de crédits, l'agrégation sera effectuée sur la **MOYENNE** (mean).*

### 3. La variable 'AMT_CREDIT'

Cette variable fait référence au montant réel du crédit octroyé et présente des valeurs manquantes.

In [48]:
# Description de la variable
previous_application["AMT_CREDIT"].describe()

count    1.670213e+06
mean     1.257411e+05
std      2.381327e+05
min      0.000000e+00
25%      0.000000e+00
50%      4.288500e+04
75%      1.350000e+05
max      4.509688e+06
Name: AMT_CREDIT, dtype: float64

In [49]:
# Nombre de crédits à zéro
credit_zero = previous_application[previous_application["AMT_CREDIT"] == 0]

credit_zero.shape[0]

634984

*Il est tout à fait normal d'avoir des annuités à zéro (crédits refusés, annulés ou non finalisés). Comme pour le jeu de données du Bureau de crédits, l'agrégation sera effectuée sur la **SOMME** (sum) et la **MOYENNE** (mean).*

*Ce jeu de données présentant des valeurs manquantes, ces dernières seront imputées après les étapes de feature engineering et avant les agrégations.*

### 4. La variable 'AMT_GOODS_PRICE'

**A NOTER**: Cette variable présente des valeurs manquantes qui seront de nouveau imputées par la valeur fictive de -2.

In [50]:
# Description de la variable
previous_application["AMT_GOODS_PRICE"].describe()

count    1.627379e+06
mean     1.150368e+05
std      2.202666e+05
min      0.000000e+00
25%      0.000000e+00
50%      4.130550e+04
75%      1.253700e+05
max      5.850000e+06
Name: AMT_GOODS_PRICE, dtype: float64

In [51]:
# Imputation par -2 pour les valeurs manquantes
previous_application["AMT_GOODS_PRICE"] = previous_application["AMT_GOODS_PRICE"].fillna(value=-2)

*L'agrégation de cette variable sera réalisée sur la **MOYENNE** (mean).*

### 5. La variable 'AMT_APPLICATION'

In [52]:
# Description de la variable
previous_application["AMT_APPLICATION"].describe()

count    1.670214e+06
mean     1.120865e+05
std      2.181828e+05
min      0.000000e+00
25%      0.000000e+00
50%      3.793500e+04
75%      1.210500e+05
max      5.850000e+06
Name: AMT_APPLICATION, dtype: float64

*La variable AMT_CREDIT est beaucoup plus intéréssante car cette dernière renseigne le montant exact du crédit, alors que AMT_APPLICATION précise le montant demandé.*

*A NOTER: Au départ, je ne souhaitais pas conserver cette variable en l'état dans le projet mais créer une nouvelle variable précisant le pourcentage du montant reçu par rapport au montant demandé. Néanmoins, ceci n'a pas été réalisé pour différentes raisons:*
- *Certains crédits acceptés possédent dans ce jeu de données une valeur de 0 pour la variable 'AMT_APPLICATION'. Par conséquent, une division par zéro renvoie des valeurs infinies...*
- *Le plus important en soit est de connaître le montant réellement octroyé.*
- *D'un point de vue métier, le mieux est de pas trop manipuler les chiffres.*

*CONCLUSION: Seule la variable AMT_CREDIT déjà traitée sera donc conservée. Comme aucun feature engineering ne sera réalisé, les valeurs manquantes peuvent donc être imputées par -2.*

**1. PREUVE DE LA PRESENCE DE VALEUR 0 POUR DES CREDITS ACCEPTES AU NIVEAU DE LA VARIABLE 'AMT_APPLICATION**

In [53]:
# Valeurs de O pour les crédits acceptés
zero_amt_application_approved = previous_application[(previous_application["AMT_APPLICATION"] == 0) & (previous_application["NAME_CONTRACT_STATUS"] == "Approved")]
zero_amt_application_approved.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
93,2522811,120854,Revolving loans,33750.0,0.0,675000.0,NaN,-2.0,THURSDAY,18,Y,1,NaN,NaN,NaN,XAP,Approved,-879,XNA,XAP,Unknown,Repeater,XNA,Cards,x-sell,Credit and cash offices,-1,XNA,0.0,XNA,Card X-Sell,-332.0,-297.0,365243.0,365243.0,365243.0,0.0
143,1975174,276237,Revolving loans,22500.0,0.0,450000.0,NaN,-2.0,TUESDAY,13,Y,1,NaN,NaN,NaN,XAP,Approved,-510,XNA,XAP,Unknown,Repeater,XNA,Cards,x-sell,Country-wide,1500,Consumer electronics,0.0,XNA,Card X-Sell,365243.0,365243.0,365243.0,365243.0,365243.0,0.0
356,1490462,449030,Revolving loans,9000.0,0.0,180000.0,NaN,-2.0,SUNDAY,13,Y,1,NaN,NaN,NaN,XAP,Approved,-691,XNA,XAP,Unknown,Repeater,XNA,Cards,x-sell,Country-wide,232,Consumer electronics,0.0,XNA,Card X-Sell,-579.0,-555.0,365243.0,-494.0,365243.0,0.0
365,2240107,323651,Revolving loans,13500.0,0.0,270000.0,NaN,-2.0,TUESDAY,12,Y,1,NaN,NaN,NaN,XAP,Approved,-650,XNA,XAP,Unknown,Repeater,XNA,Cards,x-sell,AP+ (Cash loan),5,XNA,0.0,XNA,Card X-Sell,-603.0,-577.0,365243.0,365243.0,365243.0,0.0
415,2748424,263634,Revolving loans,38250.0,0.0,765000.0,NaN,-2.0,MONDAY,12,Y,1,NaN,NaN,NaN,XAP,Approved,-680,XNA,XAP,Unknown,Repeater,XNA,Cards,x-sell,Credit and cash offices,-1,XNA,0.0,XNA,Card X-Sell,365243.0,365243.0,365243.0,365243.0,365243.0,0.0


In [54]:
# Nombre de crédits concernés
print(f'Ce cas de figure concerne {zero_amt_application_approved.shape[0]} lignes.')

Ce cas de figure concerne 45304 lignes.


**2. IMPUTATION PAR -2 POUR LES VALEURS MANQUANTES DE LA VARIABLE 'AMT_CREDIT'**

In [55]:
previous_application["AMT_CREDIT"] = previous_application["AMT_CREDIT"].fillna(value=-2)

### 6. La variable "AMT_DOWN_PAYMENT"

*Il s'agit ni plus ni moins de l'accompte qui peut-être versé en amont du prêt. En général, un acompte plus élevé peut indiquer la solvabilité d'un emprunteur, car cela signifie qu'il est capable d'économiser et de payer une partie substantielle du coût total upfront. Cela peut également réduire le risque pour le prêteur, car cela signifie que l'emprunteur a déjà investi une somme importante dans l'achat et est donc plus susceptible de respecter ses obligations de remboursement pour protéger cet investissement.*

**A NOTER**: Présence de valeurs manquantes

In [56]:
# Description de la variable
previous_application["AMT_DOWN_PAYMENT"].describe()

count    1.295814e+06
mean     3.507975e+03
std      1.425804e+04
min     -9.000000e-01
25%      0.000000e+00
50%      0.000000e+00
75%      2.475000e+03
max      3.060045e+06
Name: AMT_DOWN_PAYMENT, dtype: float64

**Nous sommes en présence de valeurs négatives (très petites), ce qui est impossible. Ces valeurs seront donc remplacées par zéro.**

**1. GESTION DES VALEURS NEGATIVES**

In [57]:
# Création d'un dataframe pour les accomptes négatifs
accompte_negatif = previous_application[previous_application["AMT_DOWN_PAYMENT"] <0]
accompte_negatif.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
368107,1284109,350530,Consumer loans,7866.090,71580.60,71581.5,-0.90,71580.60,SATURDAY,13,Y,1,-0.000014,NaN,NaN,XAP,Approved,-508,Cash through the bank,XAP,Unknown,Repeater,Mobile,POS,XNA,Country-wide,45,Connectivity,10.0,low_normal,POS mobile without interest,365243.0,-459.0,-189.0,-189.0,-185.0,0.0
1519595,1817983,133068,Consumer loans,3595.545,32719.05,32719.5,-0.45,32719.05,SUNDAY,13,Y,1,-0.000015,NaN,NaN,XAP,Approved,-478,Cash through the bank,XAP,Unknown,Repeater,Mobile,POS,XNA,Country-wide,45,Connectivity,10.0,low_normal,POS mobile without interest,365243.0,-430.0,-160.0,-430.0,-415.0,0.0


**Vu le contexte, il est tout à fait pertinent de remplacer ces valeurs par zéro.**

In [58]:
# Remplacement des valeurs négatives par zéro
for idx in accompte_negatif.index:
    previous_application.loc[idx, "AMT_DOWN_PAYMENT"] = 0

In [59]:
# Description de la variable pour vérifier la bonne imputation
previous_application["AMT_DOWN_PAYMENT"].describe()

count    1.295814e+06
mean     3.507975e+03
std      1.425804e+04
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      2.475000e+03
max      3.060045e+06
Name: AMT_DOWN_PAYMENT, dtype: float64

**2. IMPUTATION DES VALEURS MANQUANTES PAR 0 POUR LES CREDITS REFUSES, ANNULES OU NON SIGNES**

In [60]:
# Création du dataframe
refused_credit = previous_application[previous_application["NAME_CONTRACT_STATUS"] != "Approved"]
refused_credit.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
4,1784265,202054,Cash loans,0.0,0.0,0.0,0.0,0.0,THURSDAY,9,Y,1,NaN,NaN,NaN,Repairs,Refused,-781,Cash through the bank,HC,Unknown,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,-1.0
6,2315218,175704,Cash loans,0.0,0.0,0.0,0.0,0.0,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Canceled,-14,XNA,XAP,Unknown,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,-1.0
7,1656711,296299,Cash loans,0.0,0.0,0.0,0.0,0.0,MONDAY,7,Y,1,NaN,NaN,NaN,XNA,Canceled,-21,XNA,XAP,Unknown,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,-1.0
8,2367563,342292,Cash loans,0.0,0.0,0.0,0.0,0.0,MONDAY,15,Y,1,NaN,NaN,NaN,XNA,Canceled,-386,XNA,XAP,Unknown,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,-1.0
9,2579447,334349,Cash loans,0.0,0.0,0.0,0.0,0.0,SATURDAY,15,Y,1,NaN,NaN,NaN,XNA,Canceled,-57,XNA,XAP,Unknown,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,-1.0


In [61]:
# Imputation par 0 pour ces crédits
refused_credit = refused_credit.copy()

for idx in refused_credit.index:
    previous_application.loc[idx, "AMT_DOWN_PAYMENT"] = 0

**3. IMPUTATION PAR -2 POUR LES VALEURS MANQUANTES**

In [62]:
previous_application["AMT_DOWN_PAYMENT"] = previous_application["AMT_DOWN_PAYMENT"].fillna(value=-2)

*L'aggrégation sera réalisée sur **MOYENNE** (mean).*

### 7. La variable 'DAYS_DECISION'

*Cette variable est un lien temporel entre le moment où une décision a été prise concernant une précédente demande de crédit (par exemple, acceptée, refusée, en attente, etc.) et le moment où une nouvelle demande de crédit est soumise. Cette dernière peut être pertinente pour la détection d'un client à risque. L'agrégation pour cette variable sera réalisée sur la **MOYENNE** (mean).*

In [63]:
# Description de la variable
previous_application["DAYS_DECISION"].describe()

count    1.670214e+06
mean    -8.806797e+02
std      7.790997e+02
min     -2.922000e+03
25%     -1.300000e+03
50%     -5.810000e+02
75%     -2.800000e+02
max     -1.000000e+00
Name: DAYS_DECISION, dtype: float64

In [64]:
# Multiplication par -1 pour l'obtention de valeurs positives
previous_application["DAYS_DECISION"] = previous_application["DAYS_DECISION"]*-1

### 8. La variable 'CNT_PAYMENT'

*Une moyenne élevée pourrait indiquer que les clients ont généralement opté pour des crédits à long terme lors de leurs précédentes demandes, ce qui pourrait suggérer un engagement financier à plus long terme. L'agrégation sera réalisée sur la **MOYENNE** (mean).*

In [65]:
# Description de la variable
previous_application["CNT_PAYMENT"].describe()

count    1.297984e+06
mean     1.605408e+01
std      1.456729e+01
min      0.000000e+00
25%      6.000000e+00
50%      1.200000e+01
75%      2.400000e+01
max      8.400000e+01
Name: CNT_PAYMENT, dtype: float64

*Cette variable ne semble pas présenter d'anomalie et les valeurs manquantes seront remplacées par -2.*

In [66]:
# Imputation par -2 pour les valeurs manquantes
previous_application["CNT_PAYMENT"] = previous_application["CNT_PAYMENT"].fillna(value=-2)

### 9. La variable 'DAYS_FIRST_DRAWING'

*Une moyenne faible pourrait indiquer que les clients demandent souvent de nouveaux crédits peu de temps après avoir reçu des financements précédents, ce qui pourrait être un signe de dépendance au crédit. L'agrégation sera réalisée sur la **MOYENNE** (mean) si le contenu de la varible est pertinent.*

In [67]:
previous_application["DAYS_FIRST_DRAWING"].describe()

count    997149.000000
mean     342209.855039
std       88916.115834
min       -2922.000000
25%      365243.000000
50%      365243.000000
75%      365243.000000
max      365243.000000
Name: DAYS_FIRST_DRAWING, dtype: float64

**A NOTER**: 
- *Cette variable présente entre 40 et 50% de valeurs manquantes avec en plus 75% de valeurs à 365243. De ce fait, cette dernière ne sera pas utilisée.*
- *Comme je pense qu'il en sera de même pour les variables 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE' et 'DAYS_TERMINATION', un describe() sera réalisé et si nous sommes dans ce même cas de figure, ces variables ne seront pas utilisées.*

### 10. Les variables 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE' et 'DAYS_TERMINATION'

In [68]:
# Description de la variable 'DAYS_FIRST_DUE'
previous_application["DAYS_FIRST_DRAWING"].describe()

count    997149.000000
mean     342209.855039
std       88916.115834
min       -2922.000000
25%      365243.000000
50%      365243.000000
75%      365243.000000
max      365243.000000
Name: DAYS_FIRST_DRAWING, dtype: float64

*Cette variable est inutilisable (idem que précédemment).*

In [69]:
# Description de la variable 'DAYS_LAST_DUE_1ST_VERSION'
previous_application["DAYS_LAST_DUE_1ST_VERSION"].describe()

count    997149.000000
mean      33767.774054
std      106857.034789
min       -2801.000000
25%       -1242.000000
50%        -361.000000
75%         129.000000
max      365243.000000
Name: DAYS_LAST_DUE_1ST_VERSION, dtype: float64

*Cette fois-ci, nous sommes en présence de tout au plus 25% de valeurs à 365243. Il s'agit donc de rechercher le nombre exact de données à 365243.*

In [70]:
# Nombre de lignes à 365243
first_version_anormale = previous_application[previous_application["DAYS_LAST_DUE_1ST_VERSION"]==365243.000000]
first_version_anormale.shape[0]

93864

*Si nous mettons ces 93864 données en NaN, nous obtiendrions plus de 54% de valeurs manquantes ce qui est non négligeable. Etant déjà en possession de beaucoup de variables renseignant sur le fait qu'un client puisse être à risque ou non, cette variable ne sera pas utlisée.*

In [71]:
# Description de la variable 'DAYS_LAST_DUE'
previous_application["DAYS_LAST_DUE"].describe()

count    997149.000000
mean      76582.403064
std      149647.415123
min       -2889.000000
25%       -1314.000000
50%        -537.000000
75%         -74.000000
max      365243.000000
Name: DAYS_LAST_DUE, dtype: float64

In [72]:
# Nombre de lignes à 365243
last_due_anormale = previous_application[previous_application["DAYS_LAST_DUE"]==365243.000000]
last_due_anormale.shape[0]

211221

*Le nombre de lignes concerné est encore plus conséquent que la variable précédente. Par conséquent, cette variable est inutilisable.*

In [73]:
# Description de la variable 'DAYS_TERMINATION'
previous_application["DAYS_TERMINATION"].describe()

count    997149.000000
mean      81992.343838
std      153303.516729
min       -2874.000000
25%       -1270.000000
50%        -499.000000
75%         -44.000000
max      365243.000000
Name: DAYS_TERMINATION, dtype: float64

In [74]:
# Nombre de lignes à 365243
terminaison_anormale = previous_application[previous_application["DAYS_TERMINATION"]==365243.000000]
terminaison_anormale.shape[0]

225913

*Idem que précédemment.*

### 11. Les aggrégations pour ce jeu de données

**1. AGGREGATION DES DONNEES NUMERIQUES**

In [75]:
# L'agrégation en elle-même
aggregated_previous_numeriques = previous_application.groupby("SK_ID_CURR").agg({"AMT_ANNUITY":"mean", "AMT_CREDIT":["mean","sum"], "AMT_APPLICATION":"mean",
                                                                                 "AMT_GOODS_PRICE": "mean", "AMT_DOWN_PAYMENT":"mean",
                                                                              "DAYS_DECISION":"mean", "CNT_PAYMENT": "mean"}).reset_index()

aggregated_previous_numeriques.head()

SK_ID_CURR AMT_ANNUITY AMT_CREDIT            AMT_APPLICATION  \
                    mean       mean        sum            mean   
0     100001    3951.000   23787.00    23787.0        24835.50   
1     100002    9251.775  179055.00   179055.0       179055.00   
2     100003   56553.990  484191.00  1452573.0       435436.50   
3     100004    5357.250   20106.00    20106.0        24282.00   
4     100005    2406.600   20076.75    40153.5        22308.75   

  AMT_GOODS_PRICE AMT_DOWN_PAYMENT DAYS_DECISION CNT_PAYMENT  
             mean             mean          mean        mean  
0        24835.50      2520.000000        1740.0         8.0  
1       179055.00         0.000000         606.0        24.0  
2       435436.50      2294.333333        1305.0        10.0  
3        24282.00      4860.000000         815.0         4.0  
4        22308.75      2232.000000         536.0         5.0

In [76]:
# Elimination du multi-indexage
aggregated_previous_numeriques.columns = ['_'.join(col).strip() for col in aggregated_previous_numeriques.columns.values]

aggregated_previous_numeriques.head()

,SK_ID_CURR_,AMT_ANNUITY_mean,AMT_CREDIT_mean,AMT_CREDIT_sum,AMT_APPLICATION_mean,AMT_GOODS_PRICE_mean,AMT_DOWN_PAYMENT_mean,DAYS_DECISION_mean,CNT_PAYMENT_mean
0,100001,3951.000,23787.00,23787.0,24835.50,24835.50,2520.000000,1740.0,8.0
1,100002,9251.775,179055.00,179055.0,179055.00,179055.00,0.000000,606.0,24.0
2,100003,56553.990,484191.00,1452573.0,435436.50,435436.50,2294.333333,1305.0,10.0
3,100004,5357.250,20106.00,20106.0,24282.00,24282.00,4860.000000,815.0,4.0
4,100005,2406.600,20076.75,40153.5,22308.75,22308.75,2232.000000,536.0,5.0


In [77]:
# Renommage de la variable 'SK_ID_CURR_' pour permettre la jointure
aggregated_previous_numeriques.rename(columns={"SK_ID_CURR_":"SK_ID_CURR"}, inplace=True)
aggregated_previous_numeriques.head()

,SK_ID_CURR,AMT_ANNUITY_mean,AMT_CREDIT_mean,AMT_CREDIT_sum,AMT_APPLICATION_mean,AMT_GOODS_PRICE_mean,AMT_DOWN_PAYMENT_mean,DAYS_DECISION_mean,CNT_PAYMENT_mean
0,100001,3951.000,23787.00,23787.0,24835.50,24835.50,2520.000000,1740.0,8.0
1,100002,9251.775,179055.00,179055.0,179055.00,179055.00,0.000000,606.0,24.0
2,100003,56553.990,484191.00,1452573.0,435436.50,435436.50,2294.333333,1305.0,10.0
3,100004,5357.250,20106.00,20106.0,24282.00,24282.00,4860.000000,815.0,4.0
4,100005,2406.600,20076.75,40153.5,22308.75,22308.75,2232.000000,536.0,5.0


In [78]:
# Informations sur le jeu de données avant jointure
aggregated_previous_numeriques.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 338857 entries, 0 to 338856
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   SK_ID_CURR             338857 non-null  int64  
 1   AMT_ANNUITY_mean       338857 non-null  float64
 2   AMT_CREDIT_mean        338857 non-null  float64
 3   AMT_CREDIT_sum         338857 non-null  float64
 4   AMT_APPLICATION_mean   338857 non-null  float64
 5   AMT_GOODS_PRICE_mean   338857 non-null  float64
 6   AMT_DOWN_PAYMENT_mean  338857 non-null  float64
 7   DAYS_DECISION_mean     338857 non-null  float64
 8   CNT_PAYMENT_mean       338857 non-null  float64
dtypes: float64(8), int64(1)
memory usage: 23.3 MB


**Nous sommes en présence d'un jeu de données complet et entièrement numérique.**

**2. JOINTURE DES DONNES NUMERIQUES ET CATEGORIELLES**

**La jointure sera réalisée sur la variable 'SK_ID_CURR'.**

In [79]:
aggregated_previous_application = pd.merge(aggregated_previous_numeriques, aggregated_previous_categorical, on='SK_ID_CURR', how="left")
aggregated_previous_application.head()

,SK_ID_CURR,AMT_ANNUITY_mean,AMT_CREDIT_mean,AMT_CREDIT_sum,AMT_APPLICATION_mean,AMT_GOODS_PRICE_mean,AMT_DOWN_PAYMENT_mean,DAYS_DECISION_mean,CNT_PAYMENT_mean,prev_approved,prev_canceled,prev_refused,prev_unused_offer,prev_cash_loans,prev_consumer_loans,prev_revolving_loans,prev_NAME_TYPE_SUITE,prev_NFLAG_INSURED_ON_APPROVAL
0,100001,3951.000,23787.00,23787.0,24835.50,24835.50,2520.000000,1740.0,8.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0.0
1,100002,9251.775,179055.00,179055.0,179055.00,179055.00,0.000000,606.0,24.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,-1,0.0
2,100003,56553.990,484191.00,1452573.0,435436.50,435436.50,2294.333333,1305.0,10.0,3.0,0.0,0.0,0.0,1.0,2.0,0.0,1,1.0
3,100004,5357.250,20106.00,20106.0,24282.00,24282.00,4860.000000,815.0,4.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0.0
4,100005,2406.600,20076.75,40153.5,22308.75,22308.75,2232.000000,536.0,5.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,-1,-1.0


**3. DESCRIPTION ET INFORMATIONS DU JEU DE DONNEES**

In [80]:
# Informations sur le jeu de données
aggregated_previous_application.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 338857 entries, 0 to 338856
Data columns (total 18 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   SK_ID_CURR                      338857 non-null  int64  
 1   AMT_ANNUITY_mean                338857 non-null  float64
 2   AMT_CREDIT_mean                 338857 non-null  float64
 3   AMT_CREDIT_sum                  338857 non-null  float64
 4   AMT_APPLICATION_mean            338857 non-null  float64
 5   AMT_GOODS_PRICE_mean            338857 non-null  float64
 6   AMT_DOWN_PAYMENT_mean           338857 non-null  float64
 7   DAYS_DECISION_mean              338857 non-null  float64
 8   CNT_PAYMENT_mean                338857 non-null  float64
 9   prev_approved                   338857 non-null  float64
 10  prev_canceled                   338857 non-null  float64
 11  prev_refused                    338857 non-null  float64
 12  prev_unused_offe

**C'est bon, aucun client n'a été perdu en cours de route, le jeu de données est toujours complet et entièrement numérique.**

In [81]:
# Description du jeu de données
aggregated_previous_application.describe()

,SK_ID_CURR,AMT_ANNUITY_mean,AMT_CREDIT_mean,AMT_CREDIT_sum,AMT_APPLICATION_mean,AMT_GOODS_PRICE_mean,AMT_DOWN_PAYMENT_mean,DAYS_DECISION_mean,CNT_PAYMENT_mean,prev_approved,prev_canceled,prev_refused,prev_unused_offer,prev_cash_loans,prev_consumer_loans,prev_revolving_loans,prev_NAME_TYPE_SUITE,prev_NFLAG_INSURED_ON_APPROVAL
count,338857.000000,338857.000000,3.388570e+05,3.388570e+05,3.388570e+05,3.388570e+05,3.388570e+05,338857.000000,338857.000000,338857.000000,338857.000000,338857.000000,338857.000000,338857.000000,338857.000000,338857.000000,338857.000000,338857.000000
mean,278149.909581,10151.158098,1.297287e+05,6.197732e+05,1.178547e+05,1.178546e+05,3.999091e+03,919.288946,11.684307,3.059642,0.933488,0.857819,0.078015,2.207123,2.151796,0.570046,-0.470552,-0.280334
std,102879.193103,8102.821542,1.328592e+05,7.667626e+05,1.230787e+05,1.230787e+05,1.140880e+04,574.658977,7.209794,2.135404,1.723006,1.830574,0.327991,3.298588,1.820743,1.024132,0.743967,0.636467
min,100001.000000,-2.000000,0.000000e+00,0.000000e+00,0.000000e+00,-2.000000e+00,-2.000000e+00,2.000000,-2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000
25%,189061.000000,5054.247000,4.962420e+04,1.307035e+05,4.578300e+04,4.578300e+04,0.000000e+00,471.500000,6.615385,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,-1.000000,-1.000000
50%,278221.000000,8124.813750,8.965800e+04,3.225600e+05,8.157900e+04,8.157900e+04,1.242750e+03,788.250000,10.333333,3.000000,0.000000,0.000000,0.000000,1.000000,2.000000,0.000000,-1.000000,0.000000
75%,367302.000000,12794.130000,1.617705e+05,8.113725e+05,1.443690e+05,1.443690e+05,4.338000e+03,1240.666667,14.666667,4.000000,1.000000,1.000000,0.000000,3.000000,3.000000,1.000000,0.000000,0.000000
max,456255.000000,290358.000000,4.050000e+06,1.048602e+07,4.050000e+06,4.050000e+06,1.012499e+06,2922.000000,72.000000,27.000000,58.000000,68.000000,21.000000,66.000000,48.000000,31.000000,1.000000,1.000000


Le jeu de données ne semble pas présenter d'anomalie.

**4. IMPUTATION PAR LA VALEUR -2 POUR LES VALEURS NEGATIVES DES VARIABLES NUMERIQUES**

*Comme dans le premier notebook, toutes les variables présentant des valeurs négatives seront ré-imputées par la valeur -2, l'agrégation ayant pu engendrer d'autres valeurs négatives.*

**1. La variable 'AMT_ANNUITY_mean'**

In [82]:
corrected_annuity = aggregated_previous_application[(aggregated_previous_application["AMT_ANNUITY_mean"] <-0.00001)]

In [83]:
for idx in corrected_annuity.index:
    aggregated_previous_application.loc[idx, "AMT_ANNUITY_mean"] =-2

**2. La variable 'AMT_GOODS_PRICE_mean'**

In [84]:
corrected_good_price = aggregated_previous_application[(aggregated_previous_application["AMT_GOODS_PRICE_mean"] <-0.00001)]

In [85]:
for idx in corrected_good_price.index:
    aggregated_previous_application.loc[idx, "AMT_GOODS_PRICE_mean"] =-2

**3. La variable 'AMT_DOWN_PAYMENT_mean'**

In [86]:
corrected_dowm_payment = aggregated_previous_application[(aggregated_previous_application["AMT_DOWN_PAYMENT_mean"] <-0.00001)]

In [87]:
for idx in corrected_dowm_payment.index:
    aggregated_previous_application.loc[idx, "AMT_DOWN_PAYMENT_mean"] =-2

**4. La variable 'CNT_PAYMENT_mean'**

In [88]:
corrected_cnt_payment = aggregated_previous_application[(aggregated_previous_application["CNT_PAYMENT_mean"] <-0.00001)]

In [89]:
for idx in corrected_cnt_payment.index:
    aggregated_previous_application.loc[idx, "CNT_PAYMENT_mean"] =-2

**4. VERIFICATION DU JEU DE DONNEES**

In [90]:
# Description du jeu de données
aggregated_previous_application.describe()

,SK_ID_CURR,AMT_ANNUITY_mean,AMT_CREDIT_mean,AMT_CREDIT_sum,AMT_APPLICATION_mean,AMT_GOODS_PRICE_mean,AMT_DOWN_PAYMENT_mean,DAYS_DECISION_mean,CNT_PAYMENT_mean,prev_approved,prev_canceled,prev_refused,prev_unused_offer,prev_cash_loans,prev_consumer_loans,prev_revolving_loans,prev_NAME_TYPE_SUITE,prev_NFLAG_INSURED_ON_APPROVAL
count,338857.000000,338857.000000,3.388570e+05,3.388570e+05,3.388570e+05,3.388570e+05,3.388570e+05,338857.000000,338857.000000,338857.000000,338857.000000,338857.000000,338857.000000,338857.000000,338857.000000,338857.000000,338857.000000,338857.000000
mean,278149.909581,10151.158098,1.297287e+05,6.197732e+05,1.178547e+05,1.178546e+05,3.998877e+03,919.288946,11.680744,3.059642,0.933488,0.857819,0.078015,2.207123,2.151796,0.570046,-0.470552,-0.280334
std,102879.193103,8102.821542,1.328592e+05,7.667626e+05,1.230787e+05,1.230787e+05,1.140887e+04,574.658977,7.216252,2.135404,1.723006,1.830574,0.327991,3.298588,1.820743,1.024132,0.743967,0.636467
min,100001.000000,-2.000000,0.000000e+00,0.000000e+00,0.000000e+00,-2.000000e+00,-2.000000e+00,2.000000,-2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000
25%,189061.000000,5054.247000,4.962420e+04,1.307035e+05,4.578300e+04,4.578300e+04,0.000000e+00,471.500000,6.615385,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,-1.000000,-1.000000
50%,278221.000000,8124.813750,8.965800e+04,3.225600e+05,8.157900e+04,8.157900e+04,1.242750e+03,788.250000,10.333333,3.000000,0.000000,0.000000,0.000000,1.000000,2.000000,0.000000,-1.000000,0.000000
75%,367302.000000,12794.130000,1.617705e+05,8.113725e+05,1.443690e+05,1.443690e+05,4.338000e+03,1240.666667,14.666667,4.000000,1.000000,1.000000,0.000000,3.000000,3.000000,1.000000,0.000000,0.000000
max,456255.000000,290358.000000,4.050000e+06,1.048602e+07,4.050000e+06,4.050000e+06,1.012499e+06,2922.000000,72.000000,27.000000,58.000000,68.000000,21.000000,66.000000,48.000000,31.000000,1.000000,1.000000


In [91]:
# Informations sur le jeu de données
aggregated_previous_application.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 338857 entries, 0 to 338856
Data columns (total 18 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   SK_ID_CURR                      338857 non-null  int64  
 1   AMT_ANNUITY_mean                338857 non-null  float64
 2   AMT_CREDIT_mean                 338857 non-null  float64
 3   AMT_CREDIT_sum                  338857 non-null  float64
 4   AMT_APPLICATION_mean            338857 non-null  float64
 5   AMT_GOODS_PRICE_mean            338857 non-null  float64
 6   AMT_DOWN_PAYMENT_mean           338857 non-null  float64
 7   DAYS_DECISION_mean              338857 non-null  float64
 8   CNT_PAYMENT_mean                338857 non-null  float64
 9   prev_approved                   338857 non-null  float64
 10  prev_canceled                   338857 non-null  float64
 11  prev_refused                    338857 non-null  float64
 12  prev_unused_offe

In [92]:
# Sauvegarde du jeu de données
aggregated_previous_application = aggregated_previous_application.to_csv("aggregated_previous_application.csv", index = False)

### Conclusion de ce notebook
- **Conservation et agrégation des variables fiables et pertinentes pour notre projet.**
- **Temps d'exécution beaucoup trop important (plus de 7 heures) du essentiellement à la ligne de code 11.**
- **Traitement des 3 dernières tables concernant les antécédents de prêts dans la prochain notebook.**
- **Ce notebook sera nommé 'Ple_Coline_2_notebook_previous_application_092023' dans les livrables et 'notebook_2_previous_application' sur GitHub.**